# Branching and Merging I

## Git's Graph Model

A graph model defines the relationship among the commits in a repository. We will start by introducing a Directed Acyclic Graph. A graph is a way to model connected things. The actual connected things can be anything. For example, you can use a graph to model your family history or the directories and files on your computer. Graphs contain nodes connected by lines. The lines are technically called edges. Here's a graph containing three nodes and two edges. 

<img src="images/graphs.svg" width="70%" align="center"/>

The nodes represent the things that are being modeled, and the lines represent the connections between them. A directed graph means that the nodes are connected in a certain direction. The direction is represented using arrows. In the image above, we see a directed graph with three nodes and two edges. The nodes are connected with a direction pointing to the right. The arrow direction in a directed graph depends on how you define the relationship between the nodes. For example, let's say that these three nodes represent your grandparent, your parent, and you. In the image below, we can see the two directions of the graph as "Point to your child" and "Point to your parent". 

<img src="images/directed_graph.svg" width="70%" align="center"/>

So the arrow direction in a directed graph depends on how you define the relationship between the nodes. In graphs, *acyclic* means no cycles or non-circular, *i.e.*, you cannot find a path where you start and finish on the same node. The graph on the left is acyclic because you start on node `A`, move to node `B`, then move to node `C`. There is no way to go back to node `A`, once you start. The graph on the right is cyclic because you can start an end on node `A`, simply by going around the circle clockwise. 

<img src="images/acyclic_cyclic.svg" width="70%" align="center"/>

A directed acyclic graph combines the concepts that we just talked about. It contains nodes connected with arrows and has no circular paths. The graph shown here is an example of a directed acyclic graph. It has nodes connected with arrows and you cannot find a path where you start and finish on the same node. 

Git models the relationship of commits with the directed acyclic graph. The entire graph contains a project's history, where each node in Git represents a commit. The arrows point at a commit's parents. In the example below, you can see that commit `A` has two parents, commit `B` and commit `B`. The commits and the relationship between them is what forms the project's history. A branch occurs if a commit has more than one child. This graph contains a branch because commit `A` has two children, commit `B` and commit `B`. A merge occurs when a commit has more than one parent. In this example, commit `D` is a merge of branch one and branch two. 

<img src="images/dag.svg" width="35%" align="center"/>

When you use Git clients, you will often see the commit graph. Here, we see the commit graph in source tree. This commit history contains a branch and a merge. The most recent commit is at the top. Notice that the graph does not have arrows. The arrows are implied by the vertical order of the commits. Using a command line, the `git log` command with the graph option shows you this diagram. You might find that graphical clients like source tree are helpful if you want to view complex Git graphs, but the command line also works.

```shell
$ git log --oneline --graph

*   efaa6ff (HEAD -> master) Merge branch 'feature1'
|\
| * 804772b (feature1) added feature 1
|/
* 9323a1d add fileA.txt
```

## Git IDs

Internally, Git uses objects to store four types of things. 

- A **commit object** is a simple text file that contains information such as the *commit user information*, *commit message*, a *reference to the commit's parent or parents*, and a *reference to the root tree* of the project. That information is all that Git needs to rebuild the full contents of a commit. 
- An **annotated tag** is a reference to a specific commit. 
- A **tree** is an object that contains a list of the filenames and directories inside of a directory. 
- A **blob** is an object that stores the content of a file that is being managed by Git. 

A typical Git user may only interact with *commit objects* and *tags*, letting Git worry about the details related to *trees* and *blobs*. Git keeps these objects internally in something called the object store, but you typically don't directly interact with the object store. 

A **Git ID** is the name of a Git object. All the objects stored by Git are named with a 40-character hexadecimal string. These strings are commonly known as *Git IDs*, but they are also known as *object IDs*, *SHA-1s*, *hashes* and *checksums*. You commonly see these *Git IDs* as you work with Git. For example, the Git log command will show 40-character commit IDs like what we see below. 

```shell
$ git log 

  commit 048c23b5b1102a41e63fcbd32984102fc3734c66
  Author: user <user@email.com>
  Date:   Thu May 7 15:35:56 2020 -0300

  Add fileA.txt
```

That string is the name of a commit object. *Git IDs* are what's known as *secure hash algorithm* one or *SHA-1* values. The 40-character hexadecimal string is the result of a mathematical computation based on the content. Statistically speaking, the SHA-1 value is unique for a given piece of content. The exact same content will always result in the same SHA-1 value, but it is virtually impossible to find two different content files that produce the same SHA-1 value. SHA-1 values are designed to avalanche, *i.e.*, small changes in the content, leads to a large difference in the SHA-1 values. To see this, the SHA-1 of the simple string `hi`, starts with `49f6`. The SHA-1 of the same string with a trailing space `hi ` starts with `6334`. 


```shell
$ echo -n 'hi' | md5sum

  49f68a5c8493ec2c0bf489821c21fc3b

$ echo -n 'hi ' | md5sum

  63346d9b3c29ca03e984693c7200f3b8
```

You can see that even very similar content leads to drastically different SHA-1 values. The `git hash-object <file>` command can be used to create SHA-1 values from file contents. You probably won't use this command very often, but Git uses it behind the scenes to create Git IDs. First, we create a file named `fileA.txt` containing the string `hi`. We then execute the Git hash-object command, passing in `fileA.txt`. You can see that Git returns the SHA-1 of the file contents. We can execute the same command on the same file and Git will return the same SHA-1 value. The string `hi` will always return this SHA-1. Now, we can change `fileA.txt` contents to `hi ` (with a trailing space). Executing Git hash-object again, will create a drastically different SHA-1. This is the avalanching behavior that we just talked about. Small changes in content lead to large changes in SHA-1 values. 

```shell
$ echo 'hi' > fileA.txt 
$ git hash-object fileA.txt

  45b983be36b73c0788dc9cbcb76cbb80fc7bb057
  
$ echo 'hi ' > fileA.txt 
$ git hash-object fileA.txt

  0b5d6ed361eb779d2212ec820ea5568f3af7762b
```

Git hash-object is an example of a low level or [plumbing command](https://git-scm.com/book/en/v2/Git-Internals-Plumbing-and-Porcelain) in Git. You probably will not commonly use plumbing commands, but they're useful for scripting and can be useful for helping you to understand Git. Since the 40-character Git ID names are not very people friendly, Git sometimes shortens them to the first seven characters. Here you can see that the Git log command with the one line option returns a shortened Git ID for the commit object. 

```shell
$ git log --oneline

  048c23b Add fileA.txt
```

If we execute the Git log command with no options, you can see that the full 40-character version of the commit object name is used. Notice that the first seven characters match the shortened version of the Git ID. When you are entering IDs in Git commands, you can shorten the IDs too, just to use at least the first four characters when referencing an ID. If there is only one object in the object store that starts with those characters, Git will know which object you are referring to. In rare cases, two objects may start with similar characters and you may need to enter more characters of the ID in order for Git to unambiguously identify the object. In the example below, we use the `git show` command, which shows the contents of an object. Here, we include only the first four characters of the Git ID. Git knows which object you are referring to and returns information on the object. 

```shell
$ git show dc23

  commit dc233fb011d954d3417c34abeb48508f4cfb0be7
  Author: user <user@email.com>
  Date:   Wed Apr 29 16:38:09 2020 -0300

  Change fileA.txt
```


## Git References

Commits can be associated with references. A reference is a user-friendly name that points to a commit's SHA-1 value or another reference. If a reference points to another reference, it's called a symbolic reference. Here, the latest commit has two references associated with it, `HEAD` and `master`. Instead of using SHA-1 hashes and Git commands, you can use references. As an example, here we execute a `git log` command. Notice that the commit object has a SHA-1 hash and two references associated with it, `HEAD` and `master`. 

```shell
$ git log --oneline

  048c23b (HEAD -> master) Add fileA.txt
```

We could then call the `git show` command, passing it either the full SHA-1 hash, part of the SHA-1 hash, the master reference, or the HEAD reference, since they all point to the same SHA-1 hash. In this example, we passed the `HEAD` reference. Executing the command shows details related to the commit object.

```shell
$ git show HEAD

  commit 048c23b5b1102a41e63fcbd32984102fc3734c66 (HEAD -> master)
  Author: user <user@email.com>
  Date:   Thu May 7 15:35:56 2020 -0300

  Add fileA.txt
```

**Master** is the default name of the main branch in the repository. A branch is an independent path for a set of commits. If we never create a branch in a repository, then by default, all commits are part of the `master` branch. Here, we execute the `git status` command. And Git informs us that we are currently on the master branch. Any commits that we would make at this point would be to this branch.

```
$ git status

  On branch master
  Your branch is up-to-date with 'origin/master'.
```

A branch label points to the most recent commit in the branch. That commit is commonly called the "tip of the branch". Branch labels are implemented as references. In this example there are three commits on the master branch, with the master branch label pointing to the tip of the branch. Note the subtle difference between a branch and a branch label. All three commits belong to the master branch, even though the master branch label is only at the tip of the branch. Since a branch in Git is implemented as a tiny branch label, branches are extremely simple to implement and use very few resources.

<img src="images/tip_of_the_branch.svg" width="60%" align="center"/>

We mentioned that Git implements branch labels as references. These references are stored in the `.git` directory of the local repository. Just so that we can understand them better, let's take a look inside the `.git` directory and view a branch label reference. This is not a common thing to do and Git might actually optimize the directory by what is known as packing. This can change the location of the reference, so it's best not to directly work inside of this directory. Let's start by changing from the project directory to the `.git` directory. Listing the contents of the `.git` directory shows that it contains a `refs` directory. References are stored in this directory. We change to the `refs` directory. Listing the contents of this directory shows that it contains a `heads` directory and a `tags` directory. Local branch references are in the `heads` directory. We change to the `heads` directory. Listing the contents of the `heads` directory shows that there's currently just one branch label reference named `master`. It's the name of the default branch in the repository. Master is a reference which is implemented as a simple text file. If we view the contents of the master reference, we can see that it contains a SHA-1 hash. This is the SHA-1 of the commit object at the tip of the branch. This is how Git makes the association between the master branch label reference and the SHA-1 hash of the tip of the branch. Notice that reference files are tiny, containing only a short string. `HEAD` is a reference to the current commit. It usually points to the branch label of the current branch. Because it only points to the current commit and there can only be one current commit, there is only one `HEAD` reference in your local repository.

```shell
$ cd .git/
.git$ ls

  branches  COMMIT_EDITMSG  config  description  HEAD  hooks  index  info  logs  objects  refs

.git$ cd refs/
.git/refs$ ls

  heads  tags

.git/refs$ cd heads/
.git/refs/heads$ ls

  master

.git/refs/heads$ cat master 

  048c23b5b1102a41e63fcbd32984102fc3734c66
```

In the example below, we have three commits on the master branch. The `master` branch label reference points to the most recent commit. The `HEAD` reference points to the master branch label. 

<img src="images/head.svg" width="60%" align="center"/>

You can also see this relationship when you execute the `git log` command. Here we execute the `git log --oneline` to keep things concise and the `-1` option to limit the display to the current commit. You can see that for the current commit with the SHA-1 hash, starting with `483d`, there are two associated references. The HEAD reference points to the master branch label which points to the current commit's SHA-1 hash by association. You can see that this is the same relationship as the one shown in the diagram.

```shell
$ git log --oneline -1

  048c23b (HEAD -> master) Add fileA.txt
```

`HEAD` is implemented in Git as a reference. There's a single HEAD reference in a local repository and it points to the current commit. We have seen that you can view the HEAD reference by executing the `git log` command. As a learning experience, you can also view it in the `.git` directory. We start by changing from our project directory to the `.git` directory. Listing the contents of the `.git` directory shows that the `HEAD` reference is a top level file. You can view the contents of this file to see that currently `HEAD` points to the master branch label reference. When a reference points to another reference like this, it's called a symbolic reference.

```shell
$ cd .git/
.git$ ls

  branches  COMMIT_EDITMSG  config  description  HEAD  hooks  index  info  logs  objects  refs

.git$ cat HEAD

  ref: refs/heads/master
```

In Git commands, the `~` character can be appended to git IDs and references to refer to prior commits. Let's look at an example. Below, we execute the `git log` command to view the current branch's commit history. We specify the `--oneline` option to keep things concise, and the --`graph` option to show the commit graph. When we execute the command, you can see that there are four commits on this branch.

```shell
$ git log --oneline --graph
  
  * af54214 Version 2 of file A
  * c6074e1 Added fileA.txt
  * 45b983b Test fileA.txt
  * 8e129b7 Initial commit
```

If we execute `git show HEAD`, it will show the information for the current commit object, whose SHA-1 hash value begins with `af`.

```
$ git show HEAD

  (shows commit info for af54214)
```
If we append a `~` to HEAD in the command, we are referring to the current commit's parent, whose SHA-1 hash begins with `c6`. Appending a single `~` is equivalent to appending a `~1`. If we execute the command with an argument of `master~3`, we will reference the commit three commits away from the current commit, whose SHA-1 begins with `8e`. If we execute the command with a partial SHA-1, followed by three successive tildes, we again reference the commit three commits away from the current commit, whose SHA-1 begins with `8e`.

```
$ git show HEAD~ # same as HEAD~1

  (shows commit info for c6074e1)

$ git show master~3

  (shows commit info for 8e129b7)

$ git show af54214~~~

  (shows commit info for 8e129b7)
```

The caret character (`^`) can be appended to IDs and references in git commands, primarily to refer to a specific parent in a merge commit. **Merge commits** have multiple parents. As an example, `^2` refers to a merge commit's, second parent. Below, we execute the `git log` command to view a list of our commits. You can see that there are four commits, all on a branch with no merges. We execute the `git show master^` command, which refers to the first parent whose SHA-1 starts with `c6`. Note that this is the same behavior as if we had used the `~` character. We then execute `git show HEAD^2` argument. This is trying to refer to the current commit's second parent. But because there is only one parent, we receive an error. We can execute the git show command with `HEAD^^` as the argument which results in different behavior. This will return the first parent's first parent, which is the commit with the SHA-1 of `45`. Notice that this is the same behavior as if we used an argument of `HEAD~~` or `HEAD~2`.

```
$ git show master^

  (shows commit info for c6074e1)

$ git show HEAD^2

  fatal: ambiguous argument 'HEAD^2': unknown revision or path not in the working tree.

$ git show HEAD^^

  (shows commit info for 45b983b)
```

Tilda (`~`) and caret (`^`) characters can be combined arbitrarily to access any commit. Even in a tree with many branches and merges. For example `HEAD~^2` refers to the parent's second parent.

<img src="images/tilda_carret.svg" width="60%" align="center"/>

A **Tag** is a reference attached to a specific commit. It acts as a user-friendly label for the commit. There are two types of tags. The first type is a *lightweight* tag. This is implemented as a simple reference, much like a branch label or `HEAD` and refers to the commit object. The second type of tag is an *annotated tag*. This is one of the four Git object types mentioned earlier. An annotated tag is similar to a commit object in that it includes metadata such as *tag author* information, *tag date*, *tag message*, and the *ID of the commit object* referenced by the tag. 

<img src="images/tag.svg" width="60%" align="center"/>

Annotated tags can be signed and verified with GNU Privacy Guard. This enables someone like a product manager to sign off on a release, for example. In general, annotated tags are recommended over lightweight tags because they are true Git objects and offer more capabilities. To view all the tags in the repository, use the `git tag` command. Below, we execute the `git tag` command and see that this repository has two tags. One for version `0.1` and one for version `1.0`. 

```shell
$ git tag

  v0.1
  v1.0
```

Like other references, you can use tags instead of SHA-1 values in Git commands. Here we execute the git show command using a tag name as an argument. We happen to be viewing a lightweight tag, so git show shows information about the commit associated with the version `0.1` tag.

```shell
$ git tag v0.1

  commit 048c23b5b1102a41e63fcbd32984102fc3734c66 (tag: v0.1)
  (commit info)
```

To tag a commit with a lightweight tag, use the `git tag` command followed with an argument specifying the tag name. You can optionally specify a commit to be tagged. If you don't specify a commit, the commit that `HEAD` points to will be tagged. Here we execute the git tag command passing `v1.0` as a tag name. Because no commit is specified, this tags the current commit which is the commit that `HEAD` points to. We can then execute git tag to view the tag that we just created. 

```shell
$ git tag v1.0
$ git tag
  
  v1.0
```

Next, we execute the `git tag` command specifying `v0.1` as the tag name and specify the Commit as `HEAD^`. This tags the parent and the current commit with the `v0.1` tag. We execute `git tag` again, and now we see both of the lightweight tags that we created.

```shell
$ git tag v0.1 HEAD^
$ git tag
  
  v0.1
  v1.0
```

We can then use a tag name in the git show command to obtain more information on the commit referenced by the tag. To tag a commit with an annotated tag, you also use the `git tag` command, but specify the `-a` option. This will create a Git object. You must specify a tag message, which can be specify it in three ways. The first is to pass the `-m` flag specifying a message. The second is to pass the capital `F` flag specifying a file which contains the tag message. The third way is not to specify either flag, in which case an editor will be opened so that you can type the tag message. In this example, we use the `git tag` command with the `-a` option to create an annotated tag. The `-m` option is used to specify a tag message of "includes feature 2". We name the tag `v2.0`. Since we didn't specify a commit, this tag will be associated with the current commit. If you execute `git show` and specify an annotated tag as an argument, the information returned will include tag object information followed by the commit information. Below, we execute `git show` with `v2.0` as the argument.

```shell
# git tag -a [-m <msg> | -F <file>] <tagname> [<commit>] 
$ git tag -a -m 'includes feature 2' v2.0
$ git show v2.0

  tag v2.0
  Tagger: user <user@email.com>
  Date:   Wed Jun 3 15:11:07 2020 -0300

  includes feature 2

  commit af5421419802e80e6faeb0c6f0cc3211c0c78863
 (commit info)  
```

You can see that information from the annotated tag, such as the tagger name, date, and tag message is shown first. Then the information for the commit associated with the tag is displayed. The `git push` command alone does not automatically transfer tags to the remote repository. To transfer a single tag, use the `git push <remote> <tagname>` command and specify the remote name, such as `origin`, as well as the tag name. To transfer all of your tags, you can execute `git push <remote> --tags`. After you push tags, you can log into Bitbucket and view them on the remote repository. 

## Branches

All commits of a project belong to a branch. By default, commit belong to the `master` branch. A branch is a set of commits starting with the most recent commit in the branch and tracing back to the project's first commit. In the example below, starting from the most recent commit on the branch, the master branch contains commits `B` and `A`. The `featureX` branch contains commits `C`, `B` and `A`. Notice that commits `B` and `A` belong to both branches. 

<img src="images/branches_featureX.svg" width="40%" align="center"/>

### Listing and Creating Branches

Let's take a look at some of the benefits of branches. We will see that **creating a branch is fast and easy**, since a branch only creates one tiny file, a reference. Branches **enable experimentation**. Team members can isolate their work so that it doesn't impact others until the work is ready. Branches are not aware of other branches. This allows you to experiment with changes to the project, while at the same time the team retains a stable version of the project. If you have an idea for a change, you can create a branch and test your idea. Later you can throw out your branch, or merge it into the official project. Branches **enable multiple team members** to concurrently work on the project, without stepping on each other's work. Merging the work together later usually is not too difficult. Branches allow you to **support multiple versions** of the project simultaneously. For example, if the project is a software project with customers on several supported versions, you can update each version if an important hot fix is necessary. 

Branches can be **short lived** or **long running**. **Short lived** branches are commonly called topic or feature branches, and usually contain one small change to the project. For example, a topic branch may contain a new feature, a bug fix, a hot fix, a configuration change, or any other change that the project requires. When it's ready, a topic branch is merged into a long running branch. In the example below, the `featureX` branch is a topic branch. Two commits were needed to create the feature, and then the feature was merged into the master branch. 

<img src="images/short_lived.svg" width="90%" align="center"/>

**Long running** branches like the master branch live longer than topic branches, and can even live for the life of the project. Examples of branches that might live for the life of a project are a `master` branch or a `develop` branch. We will see later that git branches are flexible enough to work with many types of workflows. So you can pick a branching strategy that works best for your team and for your project. Use the `git branch` command to see a list of branches in the local repository. The branch that you are currently on has an asterisk. You can see here that there are two branches in the local repository, and the current branch is the `master` branch.

```shell
$ git branch

  dev
  * master
```

In addition to listing branches, the `git branch` command is used to create branches. You specify the new branch name as an argument to the `git branch` command. Creating a branch simply creates a new branch label reference. You remain on the original branch. Here we start with just a `master` branch containing three commits. We then create a `featureX` branch off of the most recent master commit, by creating a `featureX` branch label pointing to that commit. 

<img src="images/creating_branch.svg" width="70%" align="center"/>

To create the `featureX` branch label, we execute `git branch featureX`. We then execute `git branch`, and see that we've created the `featureX` branch, but our current branch is still the master branch. You can see in the diagram on the right that the `HEAD` reference still points to the `master` branch label after creating the `featureX` branch. 

```shell
#$ git branch <name>
$ git branch featureX
$ git branch

  dev
  featureX
  * master
```

### Check Out Branches

**Check out** is usually related to branches and does two main things. First, it switches the current commit which is the commit that the `HEAD` reference points to, to the checked out branch label or commit. In the example below, if we were on the master branch and then checked out the `featureX` branch, the `HEAD` reference changes from pointing to the `master` branch label to pointing to the `featureX` branch label. 

<img src="images/checkout.svg" width="70%" align="center"/>

The second thing that checkout does, is it updates the working tree with the files from the checked out commit. Once you have checked out a branch, you can work on and commit to the branch. In this example, the `featureX` label points to the same commit as the master branch label. So, at this point the working tree files will be the same for both branches. 

<img src="images/git_checkout.svg" width="40%" align="center"/>

Use the git checkout command to checkout a branch or commit. To checkout a branch, specify a branch name as the argument. To checkout a specific commit, specify it's SHA-1 as an argument. Here we change from the `master` branch to the `featureX` branch by executing `git checkout featureX`. We execute `git branch` and see that we're currently on the `featureX` branch. If we list the contents of the working tree, we would see that the files for the `featureX` version would be available. 

```shell
#$ git checkout <branch_or_commit>
$ git checkout featureX
$ git branch

  dev
  * featureX
  master

$ ls

  (list files of featureX)
```

To create and checkout a branch, you could first create the branch using the get branch command. After executing that command, you've created a `featureX` branch, but the `HEAD` reference is still pointing to the `master` branch label. You can then use the `git checkout` command to checkout the `featureX` branch. These two commands in combination create and switch to the `featureX` branch. To combine creating and checking out a branch into a single command, use the `-b` option with `git checkout`. This combines the `git branch` and the `git checkout` commands. This command is only for new branches. It fails if the branch already exists. 

```shell
$ git branch featureX
$ git checkout featureX

# the command below executes both commands together
$ git checkout -b featureX
$ git branch

  dev
  * featureX
  master
```

After you've created and checked out the `featureX` branch, the next commit that is made will be made to the `featureX` branch. The `HEAD` reference moves along with the `featureX` label, and the `master` label stays where it was. The `master` branch no longer points to the latest commit. In fact, the master branch doesn't know about the `featureX` branch label, or about the latest commit on `featureX`. This is how the `featureX` branch can make changes to the project without disturbing the `master` branch. 

<img src="images/new_commit.svg" width="50%" align="center"/>

Once we have checked out `master`, we can do some work in the working tree, and create a commit. Now the `featureX` branch and `master` branch both have a commit that doesn't belong to the other branch. You can see that the last commit of `featureX` uniquely belongs to the `featureX` branch, and the last commit of the `master` uniquely belongs to the `master` branch. 

<img src="images/new_commit_master.svg" width="50%" align="center"/>

Usually you checkout a branch, which checks out the commit that the branch label points to, but you also have the option of directly checking out a commit. For example, you might checkout a commit to temporarily view an older version of the project. Checking out a commit rather than a branch leads to a detached `HEAD` state. This means that instead of the `HEAD` reference pointing to a branch label, `HEAD` points directly to the SHA-1 of a commit. So a detached `HEAD` basically means that the `HEAD` reference is detached from a branch label. Git will warn you that if you continue you will be in a detached `HEAD` state. In the example below, in the before state, we have the `master` branch checked out as is normal. If we then checkout the previous commit, that commit will become the current commit, and the `HEAD` reference will point directly at that commit, creating a detached `HEAD` state. 

<img src="images/detached_head.svg" width="70%" align="center"/>

Temporarily viewing the files of a commit while in a detached `HEAD` state is perfectly fine. However, if you want to work on files of the checked out commit and create new commits, you should create a branch on that commit first. On the left, we've checked out a previous commit, creating a detached `HEAD`. If we want to create commits based off of that commit, we should first create a `featureX` branch label, and checkout that branch. That puts us back in the normal state where the `HEAD` reference points to a branch label. The next commit will then be made to the `featureX` branch. 

<img src="images/back_to_normal.svg" width="70%" align="center"/>

### Deleting a Branch

**Deleting a branch** really just means that you're deleting a branch label. Deleting a branch label normally does not delete any commits, at least not right away. This shows how truly lightweight git branches are. Use the `-d` option on the `git branch` command to delete a branch. For example, we start with two branches, but use the `-d` option, to delete the `featureX` branch. A call to `git branch` shows that the `featureX` branch has been deleted. Branch labels are commonly deleted after a topic branch has been merged. 

```shell
$ git branch

  * featureX
  master

$ git branch -d featureX

  Deleted branch featureX (was 010226b)

$ git branch

  * master
```

Let's take a look at what could happen if you tried deleting a branch label with unmerged work. Let's say that you created a `featureX` branch off of commit `C` of the master branch as shown below. Then you did some work on `featureX` and made commit `D`. At that point, if you tried to delete the `featureX` branch label, there would be a problem because commit `D` does not belong to any other branch. 

<img src="images/delete_branch.svg" width="50%" align="center"/>

Git normally will not let you do that. Here we tried to delete the `featureX` branch label. You can see that it responds by saying that the branch is not fully merged. This means that you could lose some commits that would not belong to any branch. Git tells you what you can do to really delete the branch label, which is to specify the `-D` option with git branch. 

```shell
$ git branch -d featureX

  error: The branch 'featureX' is not fully merged.
  If you are sure you want to delete it, run 'git branch -D featureX'.
```

Let's say that we don't care about commit `D` and delete the `featureX` branch label using the `-D` option. The `featureX` label will be deleted, and you're left with a dangling Committee `D`. Commit `D` doesn't belong to any branch. Git will periodically garbage collect, looking for and deleting older dangling commits. So be careful if you use the `-D` option. It assumes that you want to throw away the work that's in the dangling commits. 

Let's say that you accidentally deleted a branch label, and you left some dangling commits. You can use a command  `git reflog` to undo that. Git reflog is a command that returns a local list of recent `HEAD` commits. This list is in the local `.git` directory, but not in the repository. So this only works locally. In this case, a dangling commit `D` can be found in the list of recent `HEAD` commits. Let's look at an example. Here let's say that we use the `-D` option of `git branch`, and deleted the `featureX` branch label. This left a dangling commit. You can actually see the SHA-1 of the dangling commit in the message returned by Git. Now we will use to `git reflog` to find that SHA-1. We execute to `git reflog` command, and what is returned is a list of the recent commits that `HEAD` pointed to. You can see that the second commit in this list, is the commit that added `featureX`. You can also see that the SHA-1 of this commit matches the SHA-1 that was displayed by get earlier. Since that commit object has not been deleted from the object store, we can copy that SHA-1, and call `git checkout` with the `-b` option, creating a `featureX` branch. 

```
$ git branch -D featureX
Deleted branch featureX (was 942c36f)

$ git reflog
942c36f (HEAD -> master) HEAD@{0}: checkout: mobing from featureX to master
**434dfa0 HEAD@{1}: commit: added featureX**
942c36f (HEAD -> master) HEAD@{2}: checkout: moving from master to featureX 

$ git checkout -b featureX 434dfa0
**Switched to a new branch 'featureX'**
```

That creates a new branch label, pointing to our previously dangling commit, and we are back in business.

## Merging

**Merging** combines the work of independent branches. Usually, this involves merging a topic branch, such as the `featureX` branch, into what is called a base branch, such as the `master` branch. The base branch is usually a longer running branch than the topic branch. In the example below, the work of the two branches is combined into a single commit named `M`, which we will see later is called a merge commit. Prior to the merge, the master branch didn't know about the `featureX` branch. After the merge, the work of `featureX` is included in the `master` branch. Notice that after the merge commit, commit `B` is part of the `featureX` branch and the `master` branch. If we trace back from commit `M` to the initial commit, we can see that there's a path that includes commit `B`. If we trace the path of the `featureX` branch, we can see that it contains commits `B` and `A`. If we trace the two paths of commit `M`, we can see that all of the commits are included. In fact, we could color commit `B` half green since it belongs to both branches. For simplicity, we will keep the commits a single color, but keep in mind that a commit often belongs to more than one branch.

<img src="images/merge_commit.svg" width="40%" align="center"/>

There are four main types of merges:
- Fast-forward merge
- Merge commit
- Squash merge
- Rebase. 

We will talk about the first two now and the last two later. A **fast-forward merge** moves the base branch label to the tip of the topic branch. In the before example, on the left, we can see that we have a base branch called master and a topic branch called `featureX`. Commit `A` was made to the master branch and then a `featureX` branch was created. Commits `B` and `C` were created on the `featureX` branch. If at this point, the `featureX` branch is ready to be merged into the master branch, a fast-forward merge can be executed. As you see on the right, a fast-forward merge simply involves moving the master label from commit `A` to commit `C`. After the fast-forward merge, both branches contain exactly the same commits `C`, `B`, and `A`. All three commits are exactly the same as they were before the merge, only the master branch label was moved. The master branch now contains the work of `featureX` and the merge is complete. 

<img src="images/fast_forward_merge.svg" width="70%" align="center"/>

A fast forward merge is possible only if no other commits have been made to the base branch since the topic branch was created. If any commits have been added to the base branch, it will not allow you to perform a fast-forward merge. In the example below, a fast-forward merge is not possible because commit `D` was created after the `featureX` branch was created. If Git had allowed a fast-forward merge, the master branch label would move to commit `C`, and the work of commit `D` would be bypassed. Whoever did the work of commit `D` would not be very happy with that, so the merge is not fast-forwardable.

<img src="images/nofast_forward_merge.svg" width="35%" align="center"/>

Here are the basic steps to performing a fast-forward merge. Start by checking out the base branch, which in this case is the master branch. Then use the `git merge` command, specifying `featureX` as an argument. A fast-forward merge is the default way that Git will attempt the merge. Assuming the merge is successful, you can then delete the `featureX` branch label using the `-d` option of the `git branch` command. Thus, the steps are:

- Checkout `master` branch
- Merge `featureX` branch
- Delete `featureX` branch

Let's perform a fast-forward merge, first we use the `git log` command to view our current commit graph. We can see that there are two commits, the oldest commit includes the `master` branch label. The most recent commit includes the `feature2` branch label. Let's say that we are finished with `feature2` and are ready to merge with the master branch. The first thing we do is check out the `master` branch, the `feature2` branch will be merged into this branch. We then execute the `git merge` command, passing the `feature2` branch level as an argument. You can see that Git replied that this was a Fast-forward merge. Git will always try to perform a fast-forward merge unless you tell it not to. Now we can view our commit graph again, and see that the latest commit contains both the `master` and `feature2` labels. The fast-forward merge simply moved to the master branch label to the latest commit. We can then delete the `feature2` branch label using the `git branch` command with the `-d` option. One last view of the commit graph shows that we have a nice clean history with the `master` branch containing the work of `feature2`. 

```shell
$ git log --oneline --graph --all

  * 5046c75 (HEAD -> feature2) added feature 2
  * fbadd75 (master) added feature 1

$ git ckeckout master

  Switched to branch 'master'

$ git merge feature2
  
  Updating fbadd75..5046c75
  Fast-forward
    fileA.txt | 1 +
    1 file changed, 1 insertion(+)

$ git log --oneline --graph --all
  
  * 5046c75 (HEAD -> master, feature2) added feature 2
  * fbadd75 (master) added feature 1

$ git branch -d feature2

  Deleted branch feature2 (was 5046c75).

$ git log --oneline --graph --all
  
  * 5046c75 (HEAD -> master) added feature 2
  * fbadd75 added feature 1
```

As we have just seen, after a branch has been merged, its branch label can be deleted. This prevents a continuous increase in the number of merged branch labels as the project grows. Dealing with an ever increasing number of feature branch labels can be confusing. If you'd like to retain the knowledge of where the feature work occured, you can include this information in the feature's commit messages. Or you can add a tag that permanently marks the feature work. Even though we have deleted the merged branch label here, whether or not you should delete the branch label's after a merge is a decision that your team should make. We can see with a fast-forward merge, the resulting commit history is linear, there are no commits that have multiple parents. 

A **merge commit** combines the work of the tips of the feature and base branches and places the result into a single merge commit. In this example, the `featureX` branch was merged into the `master` branch using merge commit `M`. The commit `M` will include `featureX`, which is included in commit `C`, as well as anything else that is in commit `E`, the previous tip of the `master` branch. A merge commit always has multiple parents. We can see here that the commit `M` has two parents, commits `C` and `E`. Because of this non-linear commit graph, it's easy to see the branch in the commit history. Combining the work of multiple commits may result in what's called a merge conflict, if both branches change the same thing in different ways. 

<img src="images/merge_commit_multiple.svg" width="50%" align="center"/>

Performing a merge with a **merge commit** is basically the same as performing a **fast-forward merge**. The steps are to check out the base branch, merge the topic branch, and then optionally to delete the topic branch label. Git will automatically attempt to create a merge commit if the merge is not fast forwardable. 

- Checkout `master` branch
- Merge `featureX` branch
- Delete `featureX` branch

While executing the `git merge` command, Git will open an editor showing a default merge message. This is the commit message for the merge commit, you can accept or modify the default merge message. In this example, before the merge, you have a `featureX` branch based off of commit `A` of the master branch. Two commits, `D` and `E`, were made to the `master` branch since the `featureX` branch label was created. Because the `master` branch has moved along since the `featureX` branch commit was created, this merge is not fast-forwardable. After the merge, you have a merge commit `M` containing a combination of the `featureX` branch and the latest master branch commit `E`. All of the commits shown then belong to the `master` branch, and you have a nonlinear commit graph. Like with fast-forward merges, the feature branch label can be deleted to prevent an ever increasing number of branch labels. In this case, the nonlinear commit graph clearly shows the branches and merges. 

<img src="images/git_merge.svg" width="90%" align="center"/>


In some cases, you might have a situation where a merge is fast forwardable, but you would rather have a merge commit. For example, your team's policy might be to always merge feature branches using merge commits. This is so you can clearly see the branches in the commit graph. A no fast-forward merge means that a merge commit will always be created, even if the merge is fast-forwardable. The steps to perform a no fast-forward merge are similar to the steps for the other merges. The only exception is that you include the `--no-ff` option with the `git merge` command, this will always create a merge commit. In this case, the difference between a fast-forward merge and a merge commit is only in the project history information, not in the project files.

In the example below, we will perform a merge using a merge commit, even though the merge is fast forwardable. We will ensure that a merge commit is made using the `--no-ff` option with the `git merge` command. We start by looking at the commit graph. Notice that this graph is the same starting graph as in the previous example, with the initial commit having the `master` branch label. And the current commit having the `feature2` label. Next we check out the `master` branch so that we can begin the merge of the `feature2` branch. Next we will execute the `git merge` command, passing in the `--no-ff` option. We specified the `feature2` branch label as an argument. At this point your default Git editor will open with a default merge message of merge branch `feature2`, we will accept the default merge message. After saving the merge message Git will perform the merge. Merges are made with things called strategies which are ways to perform a merge. This strategy is called the recursive strategy.
We can then view the commit graph, you can see that the current commit is a merge commit containing the merge message that we entered. That commit has two parents, the first parent is the tip of the `feature2` branch, and the second parent is the initial commit. Optionally, you can then delete the `feature2` label. All of the commits belong to the `master` branch, so the `feature2` label is not needed.

```
$ git log --oneline --graph --all

  * 5046c75 (HEAD -> feature2) added feature 2
  * fbadd75 (master) added feature 1

$ git ckeckout master

  Switched to branch 'master'

$ git merge --no-ff feature2

  # opens editor to edit merge message
  Merge made by the 'recursive' strategy.
    fileA.txt | 1 +
    1 file changed, 1 insertion(+)

$ git log --oneline --graph --all
  *   efaa6ff (HEAD -> master) Merge branch 'feature2'
  |\
  | * 804772b (feature2) added feature 2
  |/
  * 9323a1d added feature 1

$ git branch -d feature2

  Deleted branch feature2 (was 5046c75).
```

Different teams may have different policies on what to do with fast-forwardable merges, and with merges in general. They may require a linear history to keep things clean, they may require merge commits to clearly see the branch history. Or they may let the person doing the merging decide what to do. In addition to merging topic branches into long-running branches, some teams periodically merge long-running branches into other long-running branches. These merges are usually easy, because they are fast-forwardable, even if a merge commit is used. 
Here is a review of what we've discussed in this video.